In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

# Teacher net

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
use_mps = torch.backends.mps.is_available()

device = torch.device('cpu')
batch_size = 64
test_batch_size = 1000
learning_rate = 0.01
gamma = 0.7
epochs = 5

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [6]:
dataset1 = datasets.MNIST('./data', train=True, download=True,
                          transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                          transform=transform)

In [7]:
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [8]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)

In [9]:
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), 'teacher_mnist_cnn.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316760
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.262830
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.184624
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.089464
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.045155
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.931049
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.848639
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.688510
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.681583
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.661493
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.386595
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.427317
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.235199
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.083386
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.922632
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.057137
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.032106
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.818571
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.058230
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.814337
Train Epoch: 1 [12800/60000 (

# Student net (same architecture as teacher, but smaller layers)

In [10]:
class SmallNet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3, 1)
        self.conv2 = nn.Conv2d(2, 4, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(576, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [11]:
use_mps = torch.backends.mps.is_available()

device = torch.device('cpu')
batch_size = 64
test_batch_size = 1000
learning_rate = 0.01
gamma = 0.7
epochs = 5

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset1 = datasets.MNIST('./data', train=True, download=True,
                          transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                          transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

small_model = SmallNet().to(device)
optimizer = optim.Adadelta(small_model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(small_model, device, train_loader, optimizer, epoch)
    test(small_model, device, test_loader)
    scheduler.step()

torch.save(small_model.state_dict(), 'student_mnist_cnn.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.325658
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.298236
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.298377
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.303485
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.313607
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.282637
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.306962
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.320555
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.303853
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.321808
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.315660
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.310489
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.294125
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.315456
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.308487
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.327909
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.303877
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.298004
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.302828
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.322532
Train Epoch: 1 [12800/60000 (

# Student with distillation training
Kullback-Leibler divergence loss between teacher pretrained prediction and training student + Cross entropy

In [12]:
def train_with_distillation(model, teacher_model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    teacher_model.eval()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        with torch.no_grad():
            output_teacher = teacher_model(data)

        loss = nn.KLDivLoss()(F.log_softmax(output, dim=1), F.softmax(output_teacher, dim=1)) + F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [13]:
use_mps = torch.backends.mps.is_available()

device = torch.device('cpu')
batch_size = 64
test_batch_size = 1000
learning_rate = 0.01
gamma = 0.7
epochs = 5

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset1 = datasets.MNIST('./data', train=True, download=True,
                          transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
                          transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

small_model_with_distillation = SmallNet().to(device)
optimizer = optim.Adadelta(small_model_with_distillation.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train_with_distillation(small_model_with_distillation, model, device, train_loader, optimizer, epoch)
    test(small_model_with_distillation, device, test_loader)
    scheduler.step()

torch.save(small_model_with_distillation.state_dict(), 'teacher_distilled_mnist_cnn.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.523990


/Users/dstukalov/opt/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Train Epoch: 1 [640/60000 (1%)]	Loss: 2.510278
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.520288
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.521049
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.533125
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.531584
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.509059
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.517659
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.507154
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.532525
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.517150
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.499587
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.503681
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.489764
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.474817
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.482681
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.479526
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.494006
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.493391
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.488625
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.455738
Train Epoch: 1 [13440/60

# Conclusion
Accuracy of student model with distillation increased by 11%, while training speed isn't dramatically increased.